In [1]:
import pandas as pd
import pickle
from surprise import Reader, Dataset, SVD

In [2]:
df = pd.read_csv('../dataset/preprocessed_interactions.csv')

In [3]:
df.head()

,user_id,recipe_id,date,rating,review,minutes,contributor_id,submitted,tags,n_steps,steps,ingredients,n_ingredients,calories,total_fat,sugar,sodium,protein,saturated_fat
0,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...,20,56824,2002-10-27,"['30-minutes-or-less', 'time-to-make', 'course...",5,"['blend together cake mix , oil and eggs', 'ad...","[""devil's food cake mix"", 'vegetable oil', 'eg...",4,132.3,11.0,39.0,5.0,4.0,11.0
1,76535,134728,2005-09-02,4,Very good!,40,166019,2005-08-24,"['60-minutes-or-less', 'time-to-make', 'main-i...",10,"['mix flour , salt and pepper in bowl', 'set a...","['chicken tenders', 'flour', 'garlic powder', ...",12,316.0,4.0,40.0,37.0,78.0,4.0
2,273745,134728,2005-12-22,5,Better than the real!!,40,166019,2005-08-24,"['60-minutes-or-less', 'time-to-make', 'main-i...",10,"['mix flour , salt and pepper in bowl', 'set a...","['chicken tenders', 'flour', 'garlic powder', ...",12,316.0,4.0,40.0,37.0,78.0,4.0
3,353911,134728,2006-09-26,5,Absolutely AWESOME! I was speechless when I tr...,40,166019,2005-08-24,"['60-minutes-or-less', 'time-to-make', 'main-i...",10,"['mix flour , salt and pepper in bowl', 'set a...","['chicken tenders', 'flour', 'garlic powder', ...",12,316.0,4.0,40.0,37.0,78.0,4.0
4,190375,134728,2007-03-09,5,These taste absolutely wonderful!! My son-in-...,40,166019,2005-08-24,"['60-minutes-or-less', 'time-to-make', 'main-i...",10,"['mix flour , salt and pepper in bowl', 'set a...","['chicken tenders', 'flour', 'garlic powder', ...",12,316.0,4.0,40.0,37.0,78.0,4.0


In [4]:
reader = Reader()
data = Dataset.load_from_df(df[['user_id', 'recipe_id', 'rating']], reader)

In [5]:
trainset = data.build_full_trainset()

In [6]:
algo = SVD()

In [7]:
algo.fit(trainset)

In [8]:
# save the model to disk
filename = 'svd_model.sav'
pickle.dump(algo, open(filename, 'wb'))

In [9]:
def get_recommendations(user_id, n=10):
    user_ratings = df[df.user_id == user_id]
    rated_recipes = user_ratings.recipe_id.unique().tolist()
    unrated_recipes = df[~df.recipe_id.isin(rated_recipes)].recipe_id.unique().tolist()

    testset = [[user_id, recipe_id, 0] for recipe_id in unrated_recipes]
    predictions = algo.test(testset)

    # sort predictions by estimated rating
    predictions.sort(key=lambda x: x.est, reverse=True)

    top_n = predictions[:n]

    top_n_recipes = [recipe_id for user_id, recipe_id, _, _, _ in top_n]

    # return the recipe_id from df that match the top_n_recipes
    return df[df.recipe_id.isin(top_n_recipes)].drop_duplicates(subset=['recipe_id'])

In [10]:
get_recommendations(8937)

,user_id,recipe_id,date,rating,review,minutes,contributor_id,submitted,tags,n_steps,steps,ingredients,n_ingredients,calories,total_fat,sugar,sodium,protein,saturated_fat
45096,6357,114956,2005-04-04,5,This is a wonderful salsa and one I enjoyed ma...,10,67656,2005-04-01,"['15-minutes-or-less', 'time-to-make', 'course...",3,"['in a medium bowl mix together the tomatoes ,...","['diced tomatoes', 'jalapeno', 'red onion', 'l...",8,45.8,0.0,7.0,2.0,4.0,0.0
66267,58439,31287,2009-11-29,5,Don't think this really needs another review!\...,187,29450,2002-06-14,"['weeknight', 'time-to-make', 'course', 'cuisi...",6,['make 8 vanilla ice cream balls and freeze on...,"['vanilla ice cream', 'corn flakes cereal', 'b...",5,125.6,7.0,44.0,4.0,3.0,15.0
116539,32772,58687,2003-04-08,5,I had leftover chocolate chips from another St...,30,64934,2003-04-07,"['30-minutes-or-less', 'time-to-make', 'course...",4,['insert toothpicks into the tops of the straw...,"['milk chocolate chips', 'shortening', 'strawb...",3,116.6,10.0,42.0,0.0,3.0,18.0
217691,146629,246166,2007-10-25,5,"Yum, Yum, Yum!!!!! This was the most popular ...",45,141293,2007-08-13,"['60-minutes-or-less', 'time-to-make', 'course...",6,['heat chicken and buffalo wing sauce in a ski...,"['canned chicken', 'buffalo wing sauce', 'crea...",7,277.9,33.0,9.0,21.0,21.0,42.0
348031,2001534007,60350,2017-05-12,5,dont put flour in pasta frineds,30,24386,2003-04-21,"['30-minutes-or-less', 'time-to-make', 'course...",10,['cook macaroni according to package direction...,"['elbow macaroni', 'butter', 'all-purpose flou...",7,590.7,37.0,9.0,22.0,46.0,74.0
534194,37036,45107,2004-02-15,5,This was fantastic! I made this cheese ring f...,190,31499,2002-10-31,"['weeknight', 'time-to-make', 'course', 'main-...",6,['combine all the ingrdients except preserves ...,"['sharp cheddar cheese', 'cream cheese', 'mayo...",8,189.9,19.0,33.0,6.0,13.0,29.0
677851,5060,124192,2006-10-04,5,Oh my goodness!!! How have I missed this one!!...,40,83400,2005-05-31,"['60-minutes-or-less', 'time-to-make', 'course...",12,['prepare the vegetables: peel and roughly cho...,"['potatoes', 'butternut squash', 'sweet potato...",13,742.1,74.0,41.0,53.0,41.0,104.0
885456,67656,146325,2006-05-14,5,Almost any dish with thyme and balsamic vinega...,35,248292,2005-11-25,"['60-minutes-or-less', 'time-to-make', 'course...",14,['combine first four ingredients in a shallow ...,"['all-purpose flour', 'dried thyme', 'salt', '...",11,336.5,9.0,37.0,23.0,82.0,6.0
907009,2409366,486496,2012-09-18,5,Not only does my family love these but the guy...,60,2390546,2012-09-06,"['weeknight', '60-minutes-or-less', 'time-to-m...",20,['preheat oven to 400f line a rimmed baking sh...,"['reynolds wrap foil', 'bacon', 'flakey biscui...",11,664.3,71.0,42.0,68.0,37.0,101.0
944738,225261,90674,2009-12-28,5,Lovew this French Toast!!! Easy to make and f...,22,86072,2004-05-04,"['30-minutes-or-less', 'time-to-make', 'course...",11,"['mix together the eggs , milk , flour , sugar...","['eggs', 'whole milk', 'flour', 'sugar', 'vani...",11,264.3,16.0,55.0,15.0,16.0,27.0


In [11]:
get_recommendations(353911)

,user_id,recipe_id,date,rating,review,minutes,contributor_id,submitted,tags,n_steps,steps,ingredients,n_ingredients,calories,total_fat,sugar,sodium,protein,saturated_fat
419,28649,33096,2002-07-29,5,This was very simple and very refreshing. Thi...,10,41578,2002-07-03,"['15-minutes-or-less', 'time-to-make', 'course...",4,['stir the lemonade mix with the water until d...,"['country time lemonade mix', 'water', 'vanill...",5,230.5,21.0,75.0,6.0,4.0,39.0
1916,4005,8507,2001-04-22,5,"This is a delicious, healthy and beautiful sal...",5,3288,2000-11-11,"['15-minutes-or-less', 'time-to-make', 'course...",4,['slice tomatoes and mozzarella cheese same th...,"['tomatoes', 'mozzarella cheese', 'fresh basil...",6,320.3,40.0,21.0,15.0,28.0,46.0
5839,1802640303,515167,2014-04-03,5,perfect amount of sweetness,13,1802633456,2014-03-31,"['15-minutes-or-less', 'time-to-make', 'course...",5,['first microwave simply potatoes traditional ...,['simply potatoes traditional mashed potatoes'...,5,398.6,45.0,35.0,50.0,22.0,26.0
10625,950262,319562,2008-09-09,5,"This is an incredibly good, easy, and beautifu...",30,895975,2008-08-14,"['30-minutes-or-less', 'time-to-make', 'course...",5,"['combine the mango , red pepper , scallions ,...","['mango', 'red pepper', 'scallion', 'fresh cil...",9,237.2,10.0,38.0,4.0,16.0,3.0
15129,88378,56704,2003-09-22,5,This was just about perfect! I used a pack of...,20,11176,2003-03-20,"['30-minutes-or-less', 'time-to-make', 'course...",14,['season flattened cutlets with a thin coat of...,"['chicken cutlets', 'salt and pepper', 'flour'...",12,280.8,39.0,4.0,22.0,4.0,46.0
15614,305531,241750,2007-08-07,5,Wonderful! This is so simple and the vanilla '...,60,107583,2007-07-21,"['60-minutes-or-less', 'time-to-make', 'course...",6,['allow the butter to soften at room temperatu...,"['unsalted butter', 'vanilla bean', 'splenda s...",4,103.0,17.0,0.0,0.0,0.0,36.0
16858,66246,72654,2003-10-15,5,This was a great accompaniment to the baked ha...,15,32360,2003-10-06,"['15-minutes-or-less', 'time-to-make', 'course...",6,['place all ingredients in a small saucepan in...,"['brown sugar', 'crushed pineapple', 'lemon ju...",6,268.8,1.0,255.0,6.0,1.0,0.0
26421,329769,361649,2009-10-31,5,*Reviewed for Aussie Forum recip Swap Nov 09* ...,5,461724,2009-03-19,"['15-minutes-or-less', 'time-to-make', 'course...",2,"['whisk oil , honey , lemon juice , mayonnaise...","['olive oil', 'honey', 'lemon juice', 'mayonna...",5,1106.1,140.0,304.0,14.0,2.0,62.0
27445,12790,9767,2001-07-18,5,Alright - I'm notoriously a bad cook. My prev...,55,6675,2001-06-29,"['60-minutes-or-less', 'time-to-make', 'course...",21,"['preheat oven to 250f', 'in a medium , nonsti...","['olive oil', 'red bell pepper', 'yellow bell ...",18,794.1,97.0,22.0,35.0,54.0,100.0
29112,22015,102853,2004-11-29,5,We LOVED these. I am definitely making them ...,28,165841,2004-10-28,"['30-minutes-or-less', 'time-to-make', 'course...",7,['soak the all bran cereal in the water for 5 ...,"['all-bran cereal', 'water', 'baking powder', ...",4,21.4,0.0,5.0,1.0,2.0,0.0
